# 2.1 Era-Interim

This section is about downloading Era-Interim data from the ECMWF webpage. 

ERA-Interim is a global atmospheric reanalysis from 1979 until 31 August 2019. See chapter 00, to again check what reanalysis data is. The spatial resolution of the data set is approximately 80 km on 60 vertical levels from the surface up to 0.1 hPa.  For more information, there is an open-access [journal article](https://rmets.onlinelibrary.wiley.com/doi/full/10.1002/qj.828) describing the ERA-Interim reanalysis from the Quarterly Journal of the Royal Meteorological Society. Era-Interim is now being replaced by ERA5, which produces the same type of data but has a higher spatial resolution and a more recent model. 

### 2.1.1 Registration

To be able to download any data from the ECMWF page, you will first have to register. This can easily be done [here](https://apps.ecmwf.int/registration/?back=https://www.ecmwf.int/user/login/sso?destination=en/computing/access-computing-facilities/registration-forms). Log in and accept their conditions for the use of data.

### 2.1.2 How to get the Data

There are two options to get your data: You can either retrieve it directly from the ECMWF website by navigating through their [selection tool](https://apps.ecmwf.int/datasets/data/interim-full-daily/levtype=ml/) and simply clicking on the `retrieve grib` / `retrieve netcdf` button in the end. For this you will not need any python code at all. 

Anyway, the ECMWF selection tool might not always fit your needs and you might not be able to download the data that you would like to. E.g. you cannot select more than one year of data there. So if your needs are more specific and you would like to be able to download any kind of data selection on your own in a very dynamic and easy way, some short python code will be the prefered way to go.

The general procedure for this second and more flexible option is very simple: the user writes a request in Python, submits it and retrieves a file (**grib** or **netcdf** format) including the requested data.

If your decided for the second option, follow the inctructions below.

### Preparations

Before you can write your request in Python code, there are some easy steps to be done. 

**First**, you will need to install the client library. Therefore, open a terminal window and tipe in 

    $ pip install ecmwf-api-client. 

**Second**, you have to save your API key for the ECMWF webpage in a file, so that ECMWF will know who is actually requesting the data. This works the following: 

Make sure that you are currently logged in and then find your key [here](https://pypi.org/project/ecmwf-api-client/), by clicking on "retrieve your key at". Copy paste the content of `$HOME/.ecmwfapirc` into a blank textfile. Save this file to your `$HOME` directory as `.ecmwfapirc`.
If you work on **Windows**, put the file in `C:\Users\<USERNAME>\.ecmwfapirc`. Therefore first save it as `ecmwfapirc.txt` and then rename it into `.ecmwfapirc.`, which will create a file called `.ecmwfapirc`. If you later on run your python script for retrieving the data, it will access this file in order to know your identity. 


### The Python Code request



Now, we will develope an easy way how to write a data request in Python and adjust it to your specific needs.

Before you write your python code request, have a look at the ECMWF [archive catalogue](https://apps.ecmwf.int/archive-catalogue/) to check, if the data you want to download is available. For any kind of selections the system will update the attributes in a dynamic way to reflect the current availability. So if you, for example, change the steps some of the available parameters will be added or removed.



If your data is available, you can now retrieve it! You will first copy paste some general Python code request and later on adjust this code to your own needs. Therefore go [here](https://apps.ecmwf.int/datasets/data/interim-full-daily/levtype=ml/) and select the data you need. You will not be able to select more than one year at a time here, which is maybe not satisfiying. You can adjust this later. Click on `view data retrieval request` and the python code for your selection will appear. Copy paste that one to your python script.

This will look similar to the following code: 

    #!/usr/bin/env python
    from ecmwfapi import ECMWFDataServer
    server = ECMWFDataServer()
    server.retrieve({
        "class": "ei",
        "dataset": "interim",
        "date": "19790101/19790201/19790301/19790401/19790501/19790601/19790701/19790801/19790901/19791001/19791101/19791201",
        "expver": "1",
        "grid": "0.75/0.75",
        "levtype": "sfc",
        "param": "167.128",
        "stream": "moda",
        "type": "an",
        "target": "output",
    })

Now, let's go through all of the keywords that are included in the code, see what they mean and how we can adjust them. 
You should keep the keywords that are not specified here as they were when copy pasted them from the ECMWF website.  

**date**: Those are the dates that will be downloaded. There are two ways to adjust them. The first one is to use the format `"date": "1979-01-01/to/2018-12-31"` to specify the date range you want to download. This is the simplest solution. However, this might not always work as expected. E.g. for monthly means of daily means, this format will produce an output that looks the following: 19790101/19790102/19790103/... although monthly means of daily means only needs the first of every month (wanted: 19790101/19790201/19790301/...). Therefore, the second option might be the saver one. To get a list of the dates you want to download, go to the [Archive Catalogue](https://apps.ecmwf.int/archive-catalogue/) and select the data you want. You can select various months/years there plus the parameter of interest. Finally click on `View the MARS request` and copy paste the list of dates from the output.

**stream**: This identifies the forecasting system used to generated the data when the same meteorological types are archived. In our case, `stream` is set to `moda`, which means monthly means of daily means. [Here](https://apps.ecmwf.int/codes/grib/format/mars/stream/) is a full list of all the different ECMWF streams.


**type**: Determines the type of fields to be retrieved. This keyword makes the selection between observations, images or fields. Examples of fields are: Analysis `an`, Forecast `fc`, Perturbed Forecast `pf`, Control Forecast `cf`. In our case, we will take `an` as `type` since we want to download an analysis product.

**levtype**: This is the type of your level.  Common values are: model level `ml`, pressure level `pl`, surface `sfc`, potential vorticity `pv`, potential temperature `pt` and depth `dp`. 

**levelist**: This parameter specifies the required levels. Though, you will only have to define `levelist´, if there are any levels required. If you choose the `levtype` to be `sfc` you will therefore not have to specify any `levelist`! Valid values for the `levelist` have to correspond to the selected levtype. For example, model levels can range from 1 to 91. Pressure levels are specified in hPa, e.g. 1000 or 500. 

**param**: This is the Parameter you want to download. The best solution to get your parameter code number, is to go to https://apps.ecmwf.int/datasets/data/interim-full-daily/levtype=ml/, select the desired parameter and retrieve the python code for this parameter. You can now copy paste the `param` code line. There is also a [list](http://apps.ecmwf.int/codes/grib/param-db) for the codes of each parameter, but it is fairly hard to find the right parameter there.

**target**: This is the name of the file that you will create by downloading the data. If you want to download the data in netcdf format, you should name the `target`something similar to `mydata.nc`.

**format**: This is the format of your downloaded file. Take `netcdf`if you want the data to be in netcdf format. This is the format we will use for the next chapters. This keyword is not included in the python code that you will get via copy paste from the `view data retrieval request` button of the ECMWF website. So do not forget to add it yourself if you would like to retrieve the data in netcdf format! 

Klick [here](https://confluence.ecmwf.int/display/UDOC/MARS+keywords) for more information about the keywords and how to adjust them. 





A simple example of code for the **monthly means of daily means** of the **2m temperature** and the period **1999-2000** in format `netcdf` would look like the following: 

    from ecmwfapi import ECMWFDataServer
    server = ECMWFDataServer()
    server.retrieve({
        "class": "ei",
        "dataset": "interim",
        "date": "19990101/19990201/19990301/19990401/19990501/19990601/19990701/19990801/19990901/19991001/19991101/19991201/20000101/20000201/20000301/20000401/20000501/20000601/20000701/20000801/20000901/20001001/20001101/20001201",
        "expver": "1",
        "grid": "0.75/0.75",
        "levtype": "sfc",
        "levelist": "200/500/850",
        "param": "167.128",
        "stream": "moda",
        "type": "an",
        "format": "netcdf",
        "target": "2mt_1999to2000.nc",
    })

Or the code for the **monthly means of daily means** of the **U wind** on pressure levels **850/500/200 hPa** and the period **1999-2000** in format `netcdf`:

    from ecmwfapi import ECMWFDataServer
    server = ECMWFDataServer()
    server.retrieve({
        "class": "ei",
        "dataset": "interim",
        "date": "19990101/19990201/19990301/19990401/19990501/19990601/19990701/19990801/19990901/19991001/19991101/19991201/20000101/20000201/20000301/20000401/20000501/20000601/20000701/20000801/20000901/20001001/20001101/20001201",
        "expver": "1",
        "grid": "0.75/0.75",
        "levtype": "pl",
        "levelist": "200/500/850",
        "param": "131.128",
        "stream": "moda",
        "type": "an",
        "format": "netcdf",
        "target": "U_200500850_1999to2000.nc",
    })

If you get the following Error-Message: 

`self._sslobj.do_handshake()
 SSLError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:590)`

Simply copy paste the text below to the start of your notebook. This should make the code run properly.

    import ssl
    import os

    if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
            getattr(ssl, '_create_unverified_context', None)): 
        ssl._create_default_https_context = ssl._create_unverified_context

By now you should have been able to retrieve the data you need! In the next chapter, you will lern how to work with this data and generate plots.